In [1]:
import time
import random
import numpy as np
from sklearn.linear_model import LinearRegression
import collections

import altair as alt
import pandas as pd
import plotly.express as px
alt.data_transformers.disable_max_rows()

samplesize = 500
Am, Bm, Cm, = 60, 10, 30
Xm, Ym = 1, 1
Astd,Bstd,Cstd = 11, 15, 21
Xstd,Ystd = 1, 1

m1, k1 = 1, 0
m2, k2 = 1.25, 0
m3, k3 = 0.5, 0
m4, k4 = 1.5, 0

d1 = 1
d2 = 1
d3 = 1
d4 = 1

errorA = 20
errorB = 400
errorC = 25
errorX = 25
errorY = 20

def jitter(x, scale=1):
    return x + np.random.normal(scale=scale)

def dependent(x, m, c, error=1):
    return jitter(m*x + c, scale=error) # mx +c + error


def linear(n=100):

    def next_generation(A):
        A_ = jitter(A, errorA) # previous A + error
        B_ = dependent(A_, m1, k1, error=errorB)
        C_ = dependent(B_, m2, k2, error=errorC)
        return A_, B_, C_

    def next_generation_delayed(history):
        A, _, _ = history[-1]
        A_ = jitter(A, errorA) # previous A + error
        A, _, _ = history[1 - d1]
        B_ = dependent(A, m1, k1, error=errorB)
        _, B, _ = history[1 - d2]
        C_ = dependent(B, m2, k2, error=errorC)
        return A_, B_, C_

    A = np.random.normal(loc=Am, scale=Astd) # normal distribution, Am - mean and Astd - standard deviation
    history = collections.deque(maxlen=max([d1, d2]))
    for i in range(max([d1, d2])):
        history.append(next_generation(A))

    for i in range(n):
        history.append(next_generation_delayed(history))

    return np.array(history[-1])


def radiating(n=100):
    def next_generation(B):
        B_ = jitter(B, errorB)
        A_ = dependent(B_, m1, k1, errorA)
        C_ = dependent(B_, m2, k2, errorC)
        return A_, B_, C_

    def next_generation_delayed(history):
        _, B, _ = history[-1]
        B_ = jitter(B, errorB)
        _, B, _ = history[1-d1]
        A_ = dependent(B, m1, k1, errorA)
        _, B, _ = history[1-d2]
        C_ = dependent(B, m2, k2, errorC)
        return A_, B_, C_


    B = np.random.normal(loc=Bm, scale=Bstd) # normal distribution, Bm - mean and Bstd - standard deviation
    history = collections.deque(maxlen=max([d1, d2]))

    for i in range(max([d1, d2])):
        A, B, C = next_generation(B)
        history.append((A, B, C))

    for i in range(n):
        history.append(next_generation_delayed(history))

    return np.array(history[-1])


def convergent(n=100):

    def next_generation(A, C):
        B_ = jitter(m1*A + m2*C + k1, errorB)
        A_ = jitter(A, errorA)
        C_ = jitter(C, errorC)
        return A_, B_, C_

    def next_generation_delayed(history):
        A, _, _ = history[1-d1]
        _, _, C = history[1-d2]
        B_ = jitter(m1*A + m2*C + k1, errorB)
        A, B, C = history[-1]
        A_ = jitter(A, errorA)
        C_ = jitter(C, errorC)
        return A_, B_, C_

    A = np.random.normal(loc=Am, scale=Astd) # normal distribution, Am - mean and Astd - standard deviation
    C = np.random.normal(loc=Cm, scale=Cstd) # normal distribution, Cm - mean and Cstd - standard deviation
    history = collections.deque(maxlen=max([d1, d2]))

    for i in range(max([d1, d2])):
        A, B, C = next_generation(A, C)
        history.append((A, B, C))

    for i in range(n):
        history.append(next_generation_delayed(history))

    return np.array(history[-1])


def common_cause(n=100):
    def next_generation(X):
        X = jitter(X, errorX)
        A = dependent(X, m1, k1, errorA)
        B = dependent(X, m2, k2, errorB)
        C = dependent(X, m3, k3, errorC)
        return A, B, C, X

    def next_generation_delayed(history):
        _, _, _, X = history[-1]
        X_ = jitter(X, errorX)
        _, _, _, X = history[1-d1]
        A_ = dependent(X, m1, k1, errorA)
        _, _, _, X = history[1-d2]
        B_ = dependent(X, m2, k2, errorB)
        _, _, _, X = history[1-d3]
        C_ = dependent(X, m3, k3, errorC)
        return A_, B_, C_, X_

    X = np.random.normal(loc=Xm, scale=Xstd) # normal distribution, Xm - mean and Xstd - standard deviation
    history = collections.deque(maxlen=max([d1, d2, d3]))
    for i in range(max([d1, d2, d3])):
        A, B, C, X = next_generation(X)
        history.append((A, B, C, X))

    for i in range(n):
        history.append(next_generation_delayed(history))

    A, B, C, _ = history[-1]
    return np.array([A,B,C])


def single_difference_cause(n=100):

    def next_generation(A, X):
        X = jitter(X, errorX)
        A = jitter(A, errorA)
        B = jitter(m1*A + m2*X + k1, errorB)
        C = dependent(X, m3, k3, errorC)
        return A, B, C, X

    def next_generation_delayed(history):
        A, _, _, X = history[-1]
        A_ = jitter(A, errorA)
        X_ = jitter(X, errorX)

        A, _, _, _ = history[1-d1]
        _, _, _, X = history[1-d2]
        B_ = jitter(m1*A + m2*X + k1, errorB)

        _, _, _, X = history[1-d3]
        C_ = dependent(X, m2, k3, errorC)

        return A_, B_ , C_, X_

    X = np.random.normal(loc=Xm, scale=Xstd) # normal distribution, Xm - mean and Xstd - standard deviation
    A = np.random.normal(loc=Am, scale=Astd) # normal distribution, Am - mean and Astd - standard deviation
    history = collections.deque(maxlen=max([d1, d2, d3]))
    for i in range(max(d1, d2, d3)):
        A,B,C,X = next_generation(A,X)
        history.append((A, B, C, X))

    for i in range(n):
        history.append(next_generation_delayed(history))

    A, B, C, X = history[-1]
    return np.array([A, B, C])


def double_difference_cause(n=100):
    def next_generation(X,Y):
        X = jitter(X, errorX)
        Y = jitter(Y, errorY)
        A = dependent(X, m1, k1, errorA)
        B = jitter(m2*X+m3*Y+k2, errorB)
        C = dependent(Y, m4, k4, errorC)
        return A, B, C, X, Y

    def next_generation_delayed(history):
        _, _, _, X, Y = history[-1]
        X_ = jitter(X, errorX)
        Y_ = jitter(Y, errorY)

        _, _, _, X, _ = history[1-d1]
        A_ = dependent(X, m1, k1, errorA)

        _, _, _, X, _ = history[1-d2]
        _, _, _, _, Y = history[1-d3]
        B_ = jitter(m2*X+m3*Y+k2, errorB)

        _, _, _, _, Y = history[1-d4]
        C_ = dependent(Y, m4, k4, errorC)
        return A_, B_, C_, X_, Y_


    X = np.random.normal(loc=Xm, scale=Xstd) # normal distribution, Xm - mean and Xstd - standard deviation
    Y = np.random.normal(loc=Ym, scale=Ystd) # normal distribution, Ym - mean and Ystd - standard deviation
    history = collections.deque(maxlen=max([d1, d2, d3, d4]))

    for i in range(max([d1, d2, d3, d4])):
        A, B, C, X, Y = next_generation(X, Y)
        history.append((A, B, C, X, Y))

    for i in range(n):
        history.append(next_generation_delayed(history))

    A, B, C, X, Y = history[-1]
    return np.array([A, B, C])


def simulations_data(pathway, n=1000):
    random.seed(time.time())
    return np.array([pathway() for i in range(n)])


def regress(X,Y):
    model = LinearRegression()
    mXY = model.fit(X.reshape(-1,1), Y)
    r_sqr = mXY.score(X.reshape(-1,1), Y)
    residual = Y - model.predict(X.reshape(-1, 1))
    return mXY.intercept_, mXY.coef_[0], r_sqr, residual


def get_slope_intercept(model):
    return model._slopt, model._intercept

def compute_regresssion(ABC):
    A,B,C = ABC.transpose()
    RAB = regress(A, B)
    RBC = regress(B, C)
    RAC = regress(A, C)
    corrE = np.corrcoef(np.array([RAB[3], RBC[3]]))
    corrE_BA_C = np.corrcoef(np.array([np.square(RAB[3]), C]))


    #print(RAB[1]*RBC[1]-RAC[1]) ## better to look at distribution of this error..it should come with center as 0
    return {"kAB": RAB[0], "kBC":RBC[0], "kAC":RAC[0],
            "mAB":RAB[1], "mBC":RBC[1], "mAC":RAC[1],
            "r_sqrAB":RAB[2], "r_sqrBC":RBC[2], "r_sqrAC":RAC[2],
            "r_E":corrE[0,1],
            "r_E_BA_C":corrE_BA_C[0,1],
            "n":len(A)}

def compute_correlation(ABC):
    corr = np.corrcoef(ABC.transpose())
    rAB, rBC, rAC = corr[0,1], corr[1,2], corr[0,2]
    #print(rAB**2*rBC**2-rAC**2) ## better to look at distribution of this error..it should come with center as 0
                                ## or see correlation between these two quantities should be 1 and if we regress ,
                                ###       it should have slope 1
    return {"rAB":rAB, "rBC":rBC, "rAC":rAC}


def overall_simulation(n=100):
    stats = []
    data = []
    for i in range(n):
        ABC = simulations_data(convergent, samplesize) # you can change pathway here
        r = compute_regresssion(ABC)
        r.update(compute_correlation(ABC))
        stats.append(r)
        data.append(ABC)
    return pd.DataFrame(stats), np.array(data)

d, ABC_all = overall_simulation()

In [2]:
def compute_confidence_interval(r, n):
    def boundary(zeta):
        return ((np.exp(2*zeta))-1)/((np.exp(2*zeta))+1)

    z = 0.5*np.log(((1-r)/(1+r)))
    zetal = z-1.96*np.sqrt(1/(n-3))
    rl = boundary(zetal)
    zetau = z+1.96*np.sqrt(1/(n-3))
    ru = boundary(zetau)
    return rl, ru

def confidence_status(L, U, v):
    l = v.where(v < L, "less")
    l.where(v > U, "more", inplace=True)
    w = l.where((v>= L) & (v <= U), "within")
    return w

def confidence_status_(r, n):
    L, U = compute_confidence_interval(r, n)
    return confidence_status(L, U, r)


def add_confidence_stats(d):
    d['rAB2*rBC2-rAC2']=d.rAB**2 * d.rBC**2 - d.rAC**2
    d['r_E_BA_C2-rBC2'] = d.r_E_BA_C**2 - d.rBC**2
    #d['rAC2'] = d.rAC**2
    d['mAB*mBC-mAC'] = d.mAB*d.mBC - d.mAC
    L,U = compute_confidence_interval(d.rAC**2, d['n'])
    d['confidence'] = confidence_status(L, U, d.rAB**2*d.rBC**2)
    L,U = compute_confidence_interval(d.rBC**2, d['n'])
    d['confidence_residual_corr'] = confidence_status(L, U, d.r_E)
    L,U = compute_confidence_interval(d.r_E, d['n'])#???
    d['confidence_corrected_bc_corr'] = confidence_status(L, U, d.r_E_BA_C**2)

def confidence_graphs(d):
    confidence = alt.Chart(d, title=f"Correlation confidence{(d['confidence']=='within').sum()}/{len(d)}").mark_point().encode(
        x = alt.X('rAB2:Q'),
        y = alt.Y('rBC2:Q'),
        color = alt.Color('confidence:N',
                          scale=alt.Scale(domain=['less', 'within', 'more'],
                                          range=['orange', 'green','red'])),
    ).transform_calculate(
        rAB2='datum.rAB*datum.rAB',
        rBC2='datum.rBC*datum.rBC'
    )
    
    confidence_res_corr = alt.Chart(d, title=f"residual corr {(d['confidence_residual_corr']=='within').sum()}/{len(d)}").mark_point().encode(
        x = alt.X('rAB2:Q'),
        y = alt.Y('rBC2:Q'),
        color = alt.Color('confidence_residual_corr:N',
                          scale=alt.Scale(domain=['less', 'within', 'more'],
                                          range=['orange', 'green','red'])),
    ).transform_calculate(
        rAB2='datum.rAB*datum.rAB',
        rBC2='datum.rBC*datum.rBC'
    )
    
    confidence_corrected_bc_corr = alt.Chart(d, title=f"corrected bc corr {(d['confidence_corrected_bc_corr']=='within').sum()}/{len(d)}").mark_point().encode(
        x = alt.X('rAB2:Q'),
        y = alt.Y('rBC2:Q'),
        color = alt.Color('confidence_corrected_bc_corr:N',
                          scale=alt.Scale(domain=['less', 'within', 'more'],
                                          range=['orange', 'green','red'])),
    ).transform_calculate(
        rAB2='datum.rAB*datum.rAB',
        rBC2='datum.rBC*datum.rBC'
    )
    
    return alt.vconcat(confidence,confidence_res_corr, confidence_corrected_bc_corr)

add_confidence_stats(d)
confidence_graphs(d)

alt.VConcatChart(...)

In [7]:
def stats_graphs(d):
    d['rAB2*rBC2-rAC2']=d.rAB**2 * d.rBC**2 - d.rAC**2
    d['r_E_BA_C2-rBC2'] = d.r_E_BA_C**2 - d.rBC**2
    #d['rAC2'] = d.rAC**2
    d['mAB*mBC-mAC'] = d.mAB*d.mBC - d.mAC
    L,U = compute_confidence_interval(d.rAC**2, d['n'])
    d['confidence'] = confidence_status(L, U, d.rAB**2*d.rBC**2)
    slope_histogram = alt.Chart(d).mark_bar().encode(
        x=alt.X('mAB*mBC-mAC:Q', bin=True),
        y='count()').properties(title="slope diff histogram")

    bincount = 100
    ticks = 10
    correlation_graph = alt.Chart(d).mark_bar().encode(
        x=alt.X('rAB2*rBC2-rAC2:Q',bin=True,
                   axis=alt.Axis(
                    tickCount=ticks,
                    grid=False)),
        y='count()').properties(
            title="Correlation")
    residual_correlation = alt.Chart(d).mark_bar().encode(
        x=alt.X('r_E:Q', bin=True),
        y='count()').properties(title="Correlation of residuals")
    corrected_correlation = alt.Chart(d).mark_bar().encode(
        x=alt.X("r_E_BA_C2-rBC2:Q", bin=True),
        y='count()').properties(
        title="Corrected Correlation")

    return alt.vconcat(slope_histogram,correlation_graph,residual_correlation, corrected_correlation)

stats_graphs(d)

alt.VConcatChart(...)

In [4]:
A, B, C = random.choice(ABC_all).transpose()
AB = alt.Chart(pd.DataFrame({"A":A, "B":B})).mark_circle().encode(
    x="A",
    y="B")
BC= alt.Chart(pd.DataFrame({"B":B, "C":C})).mark_circle().encode(
    x="B",
    y="C")
AC = alt.Chart(pd.DataFrame({"A":A, "C":C})).mark_circle().encode(
    x="A",
    y="C")

alt.vconcat(AB, BC, AC)

alt.VConcatChart(...)

In [5]:
d

,kAB,kBC,kAC,mAB,mBC,mAC,r_sqrAB,r_sqrBC,r_sqrAC,r_E,...,n,rAB,rBC,rAC,rAB2*rBC2-rAC2,r_E_BA_C2-rBC2,mAB*mBC-mAC,confidence,confidence_residual_corr,confidence_corrected_bc_corr
0,60.986119,10.426471,43.894387,1.057241,0.264868,0.032875,0.143780,0.319967,0.000634,0.094071,...,500,0.379183,0.565657,0.025181,0.045371,-0.319678,0.247154,more,within,within
1,50.041120,-5.942547,21.368196,0.989998,0.303262,0.071519,0.151575,0.367397,0.003160,0.095534,...,500,0.389327,0.606133,0.056215,0.052528,-0.365128,0.228710,more,within,within
2,68.696573,0.169750,38.365249,0.740112,0.274896,-0.081401,0.075697,0.340468,0.004126,0.079204,...,500,0.275131,0.583496,-0.064230,0.021647,-0.336885,0.284854,more,within,more
3,22.614876,-11.417505,10.683778,0.983168,0.268016,0.021983,0.137498,0.328946,0.000315,0.095010,...,500,0.370808,0.573538,0.017742,0.044915,-0.328874,0.241522,more,within,within
4,17.131783,5.492038,23.313148,1.045506,0.258325,0.027088,0.148061,0.323742,0.000482,0.099857,...,500,0.384787,0.568983,0.021958,0.047451,-0.318126,0.242992,more,within,within
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,47.997409,2.488018,27.946567,0.982086,0.244050,0.040479,0.136709,0.299012,0.001166,0.079867,...,500,0.369742,0.546820,0.034146,0.039712,-0.298713,0.199199,more,within,more
96,23.216818,5.343501,33.520496,1.027382,0.267270,-0.074791,0.161721,0.305996,0.003671,0.149233,...,500,0.402145,0.553169,-0.060591,0.045815,-0.305996,0.349379,more,within,within
97,19.094138,10.830696,29.311326,1.049094,0.237905,0.012635,0.160673,0.298691,0.000123,0.108660,...,500,0.400840,0.546526,0.011090,0.047868,-0.298362,0.236949,more,within,within
98,48.600120,2.719789,36.634267,1.022539,0.291111,0.003256,0.139767,0.342114,0.000006,0.107481,...,500,0.373855,0.584905,0.002392,0.047811,-0.342067,0.294416,more,within,within
